In [1]:
import pandas as pd 
import numpy as np
#import spacy
#from spacy.lang.en.stop_words import STOP_WORDS
from bs4 import BeautifulSoup
import re

from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [2]:
#Reading the dataset

df1 = pd.read_excel('SENSEX COMPANIES NEWS DATA.xlsx', 'SENSEX NEWS DATA')
df1.head()

,Uniqueguid,CompanyID,CompanyName,Last_Fetch,Title,Data
0,B9A7277A-7D1D-4532-9AE2-9C737AB11005,100221,ASIAN PAINTS LTD,2019-01-04,ASIANPAINT : Asian Paints Limited Financial Re...,N
1,AE936134-F927-4AC9-813E-9DFA86FC3C3C,100221,ASIAN PAINTS LTD,2019-01-04,Asian Paints Ltd - 500820 - Board Meeting Inti...,N
2,21FA6CD4-6EE2-4650-918C-162B2C8E6B52,100221,ASIAN PAINTS LTD,2019-01-07,Asian Paints Ltd - 500820 - Announcement under...,N
3,31FAB681-7D05-490C-ABF2-DBAFDE72ADC4,100221,ASIAN PAINTS LTD,2019-01-09,ASIAN PAINTS LTD. - 500820 - Disclosures under...,N
4,023C7785-FCA9-49F5-B004-73B1390C204F,100221,ASIAN PAINTS LTD,2019-01-21,3 insurance firms Asian Paints results,N


In [3]:
df1.shape

(887956, 6)

In [4]:
df2 = pd.read_excel('SENSEX COMPANIES NEWS DATA.xlsx', 'SENSEX TWITTER DATA')
df2.head()

,uniqueguid,DimValue,CompanyName,Last_Fetch,Title,Data
0,F0829030-16A9-4C2B-BCF0-1F1F7819826F,102915,WIPRO LTD,2020-10-03,Dr Shravan Subramanyam named as new MD Preside...,T
1,509303C1-D3C0-4204-A149-67EDE36CFEDD,102915,WIPRO LTD,2020-10-04,ETTelecom | IT firms may see revival rebound i...,T
2,74FA45A1-32E8-4FB5-8341-32D537A80B11,102915,WIPRO LTD,2020-10-05,Tech majors IBM Wipro and Dell had allegedly f...,T
3,6180EDA0-26FC-48BC-AA51-0A9CF2D8E263,102915,WIPRO LTD,2020-10-05,#CNBCTV18Market | Wipro surges to a 20-year hi...,T
4,648B6588-27A5-498E-A10F-7A5FB84E73BB,102915,WIPRO LTD,2020-10-05,Wipro trades at 52-week high; Top gainer on Ni...,T


In [5]:
df2.shape

(123700, 6)

In [6]:
df2.rename(columns={'DimValue':'CompanyID'},inplace=True)

In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123700 entries, 0 to 123699
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   uniqueguid   123700 non-null  object        
 1   CompanyID    123700 non-null  int64         
 2   CompanyName  123700 non-null  object        
 3   Last_Fetch   123700 non-null  datetime64[ns]
 4   Title        123680 non-null  object        
 5   Data         123700 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 5.7+ MB


In [8]:
df2['Last_Fetch']=df2['Last_Fetch'].dt.strftime('%Y%m%d')

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123700 entries, 0 to 123699
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uniqueguid   123700 non-null  object
 1   CompanyID    123700 non-null  int64 
 2   CompanyName  123700 non-null  object
 3   Last_Fetch   123700 non-null  object
 4   Title        123680 non-null  object
 5   Data         123700 non-null  object
dtypes: int64(1), object(5)
memory usage: 5.7+ MB


In [10]:
df2

,uniqueguid,CompanyID,CompanyName,Last_Fetch,Title,Data
0,F0829030-16A9-4C2B-BCF0-1F1F7819826F,102915,WIPRO LTD,20201003,Dr Shravan Subramanyam named as new MD Preside...,T
1,509303C1-D3C0-4204-A149-67EDE36CFEDD,102915,WIPRO LTD,20201004,ETTelecom | IT firms may see revival rebound i...,T
2,74FA45A1-32E8-4FB5-8341-32D537A80B11,102915,WIPRO LTD,20201005,Tech majors IBM Wipro and Dell had allegedly f...,T
3,6180EDA0-26FC-48BC-AA51-0A9CF2D8E263,102915,WIPRO LTD,20201005,#CNBCTV18Market | Wipro surges to a 20-year hi...,T
4,648B6588-27A5-498E-A10F-7A5FB84E73BB,102915,WIPRO LTD,20201005,Wipro trades at 52-week high; Top gainer on Ni...,T
...,...,...,...,...,...,...
123695,F9DDCD79-3EBC-4031-AD19-8762D89364EF,100221,ASIAN PAINTS LTD,20230120,Why is HUL slipping in trade despite earnings ...,T
123696,BB24FDF0-4F4F-4EE5-9B4D-B8C7C8BB142E,100221,ASIAN PAINTS LTD,20230120,#EarningsWithETNOW | Nifty closes below 20-EMA...,T
123697,AC724309-F1D1-4C0E-B90E-4B5D6CC4DC61,100221,ASIAN PAINTS LTD,20230120,#MarketsWithETNOW | Nifty closes below 20-EMA;...,T
123698,0064894F-02F1-4C01-9B85-E5581EA82DFB,100221,ASIAN PAINTS LTD,20230120,#MarketThisWeek | Adani Ent Bajaj Finserv Asia...,T


In [11]:
Null_Values=pd.DataFrame([(df2.isnull().sum()/df2.shape[0]*100).sort_values(ascending=False).round(2).astype('str')+'%'])

In [12]:
Null_Values.T

,0
Title,0.02%
uniqueguid,0.0%
CompanyID,0.0%
CompanyName,0.0%
Last_Fetch,0.0%
Data,0.0%


In [13]:
df2 = df2.dropna()

In [14]:
df2['Title']= df2['Title'].astype(str)

<ipython-input-14-eca3d962d4e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Title']= df2['Title'].astype(str)


In [15]:
df2

,uniqueguid,CompanyID,CompanyName,Last_Fetch,Title,Data
0,F0829030-16A9-4C2B-BCF0-1F1F7819826F,102915,WIPRO LTD,20201003,Dr Shravan Subramanyam named as new MD Preside...,T
1,509303C1-D3C0-4204-A149-67EDE36CFEDD,102915,WIPRO LTD,20201004,ETTelecom | IT firms may see revival rebound i...,T
2,74FA45A1-32E8-4FB5-8341-32D537A80B11,102915,WIPRO LTD,20201005,Tech majors IBM Wipro and Dell had allegedly f...,T
3,6180EDA0-26FC-48BC-AA51-0A9CF2D8E263,102915,WIPRO LTD,20201005,#CNBCTV18Market | Wipro surges to a 20-year hi...,T
4,648B6588-27A5-498E-A10F-7A5FB84E73BB,102915,WIPRO LTD,20201005,Wipro trades at 52-week high; Top gainer on Ni...,T
...,...,...,...,...,...,...
123695,F9DDCD79-3EBC-4031-AD19-8762D89364EF,100221,ASIAN PAINTS LTD,20230120,Why is HUL slipping in trade despite earnings ...,T
123696,BB24FDF0-4F4F-4EE5-9B4D-B8C7C8BB142E,100221,ASIAN PAINTS LTD,20230120,#EarningsWithETNOW | Nifty closes below 20-EMA...,T
123697,AC724309-F1D1-4C0E-B90E-4B5D6CC4DC61,100221,ASIAN PAINTS LTD,20230120,#MarketsWithETNOW | Nifty closes below 20-EMA;...,T
123698,0064894F-02F1-4C01-9B85-E5581EA82DFB,100221,ASIAN PAINTS LTD,20230120,#MarketThisWeek | Adani Ent Bajaj Finserv Asia...,T


In [16]:
df2_clean=df2.groupby(by=['CompanyName','Last_Fetch'])['Title'].apply(' '.join).reset_index()

In [17]:
df2_clean_Asian_Paints=df2_clean.loc[df2_clean['CompanyName']=='ASIAN PAINTS LTD']

In [18]:
df2_clean_Asian_Paints

,CompanyName,Last_Fetch,Title
0,ASIAN PAINTS LTD,20201005,Buy Asian Paints target price Rs 2140: Motila...
1,ASIAN PAINTS LTD,20201006,Market gains in the opening session: HDFC Asia...
2,ASIAN PAINTS LTD,20201007,"Technical Update & TOP Pick! ""Technical Pick: ..."
3,ASIAN PAINTS LTD,20201008,"Technical Update & TOP Pick!""Technical Pick: A..."
4,ASIAN PAINTS LTD,20201009,"Imp results next fortnight"" ""tues 13th – Wipro..."
...,...,...,...
429,ASIAN PAINTS LTD,20230115,Q3 results this week: ICICI Bank HUL Asian Pai...
430,ASIAN PAINTS LTD,20230116,"Imp week of earnings""16th jan monday: federal ..."
431,ASIAN PAINTS LTD,20230118,#EarningsWithETNOW | Asian Paints expected to ...
432,ASIAN PAINTS LTD,20230119,???? Asian Paint Hindustan Zinc ?? Mahindra Li...


In [19]:
def preprocess_text(texts):
    texts = texts.lower() 
    texts = re.sub(r'[^\x00-\x7F]+',' ', texts) 
    splitwords = texts.split()
    splitwords = filter(lambda x: x[0]!= '@' , texts.split()) 
    splitwords = [word for word in splitwords if word not in set(stopwords.words('english'))] 
    texts = " ".join(splitwords)
    return texts

In [20]:
from nltk.corpus import stopwords

In [21]:
df2_clean_Asian_Paints['Title'] =df2_clean_Asian_Paints.Title.apply(preprocess_text)

<ipython-input-21-2a3df0e5f3d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['Title'] =df2_clean_Asian_Paints.Title.apply(preprocess_text)


In [22]:
df2_clean_Asian_Paints['Title'][0]

'buy asian paints target price rs 2140: motilal oswal #etmarkets #dailyrecos #stockpicks #dailypicks asian paints\' home cor foray add sheen near-term volume growth""@imharshaj "" https://t.co/guon6cki1t asian paints ltd consider q2 h1 fy21 results interim dividend oct 22 2020""@asianpaints #q2fy21 #h1fy21 #results #interimdividend "" https://t.co/nfeaopmxnu'

In [23]:
df2_clean_Asian_Paints['Title'] = df2_clean_Asian_Paints['Title'].apply(lambda i : re.sub(r"\D(\d{6})\D",'', i))

<ipython-input-23-5fc00c78f4e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['Title'] = df2_clean_Asian_Paints['Title'].apply(lambda i : re.sub(r"\D(\d{6})\D",'', i))


In [24]:
df2_clean_Asian_Paints['Title'] = df2_clean_Asian_Paints['Title'].apply(lambda i : re.sub(r'([A-Za-z0-9+_]+@[A-Za-z0-9+_]+\.[A-Za-z0-9+_]+)','', i))

<ipython-input-24-990fb2bdf23b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['Title'] = df2_clean_Asian_Paints['Title'].apply(lambda i : re.sub(r'([A-Za-z0-9+_]+@[A-Za-z0-9+_]+\.[A-Za-z0-9+_]+)','', i))


In [25]:
df2_clean_Asian_Paints['Title'] = df2_clean_Asian_Paints['Title'].apply(lambda i : re.sub(r'https?://[^\s<>"]+|www\.[^\s<>"]+','', i))

<ipython-input-25-607f81e14ee5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['Title'] = df2_clean_Asian_Paints['Title'].apply(lambda i : re.sub(r'https?://[^\s<>"]+|www\.[^\s<>"]+','', i))


In [26]:
df2_clean_Asian_Paints['Title']= df2_clean_Asian_Paints['Title'].apply(lambda d : re.sub('[^A-Z a-z 0-9-]+','', d))

<ipython-input-26-75ba91cce1fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['Title']= df2_clean_Asian_Paints['Title'].apply(lambda d : re.sub('[^A-Z a-z 0-9-]+','', d))


In [27]:
# Remove multiple spaces 

df2_clean_Asian_Paints['Title'] = df2_clean_Asian_Paints['Title'].apply(lambda d : " ".join(d.split()))

<ipython-input-27-133cab9d6905>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['Title'] = df2_clean_Asian_Paints['Title'].apply(lambda d : " ".join(d.split()))


In [28]:
pip install num2words

Note: you may need to restart the kernel to use updated packages.


In [29]:
from num2words import num2words

In [30]:
def num_to_words(text):
    '''
    Convert Numbers to Words
    
    Args:
        text (str): String to which the function is to be applied, string
    
    Returns:
        Clean string with converted Numbers to Words
    ''' 
    after_spliting = text.split()

    for index in range(len(after_spliting)):
        if after_spliting[index].isdigit():
            after_spliting[index] = num2words(after_spliting[index])
    numbers_to_words = ' '.join(after_spliting)
    return numbers_to_words

In [31]:
df2_clean_Asian_Paints['Title'] =df2_clean_Asian_Paints.Title.apply(num_to_words)

<ipython-input-31-28c80ffb10a1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['Title'] =df2_clean_Asian_Paints.Title.apply(num_to_words)


In [32]:
df2_clean_Asian_Paints['Title'][0]

'buy asian paints target price rs two thousand, one hundred and forty motilal oswal etmarkets dailyrecos stockpicks dailypicks asian paints home cor foray add sheen near-term volume growthimharshaj asian paints ltd consider q2 h1 fy21 results interim dividend oct twenty-two 2020asianpaints q2fy21 h1fy21 results interimdividend'

In [33]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

classifier2 = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment")

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [34]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
# This model only exists in PyTorch, so we use the `from_pt` flag to import that model in TensorFlow.
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)
classifier2 = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [35]:
df2_clean_Asian_Paints['Title'][0]

'buy asian paints target price rs two thousand, one hundred and forty motilal oswal etmarkets dailyrecos stockpicks dailypicks asian paints home cor foray add sheen near-term volume growthimharshaj asian paints ltd consider q2 h1 fy21 results interim dividend oct twenty-two 2020asianpaints q2fy21 h1fy21 results interimdividend'

In [36]:
classifier2('buy asian paints target price rs two thousand, one hundred and forty motilal oswal etmarkets dailyrecos stockpicks dailypicks asian paints home cor foray add sheen near-term volume growthimharshaj asian paints ltd consider q2 h1 fy21 results interim dividend oct twenty-two 2020asianpaints q2fy21 h1fy21 results interimdividend')

[{'label': '1 star', 'score': 0.3143950402736664}]

In [37]:
#texts_pretrained = df1_clean_Asian_Paints['Title'].tolist()
level = []
overall_twitter_score= []

In [38]:
def scoring(title):
    for text in title:
        score_list = classifier2(text)
        #print(score_list)
        for value in score_list:
            level.append(value['label'])
            overall_twitter_score.append(value['score'])
            break
        break

In [39]:
for i in range(0, len(df2_clean_Asian_Paints)):
    scoring(df2_clean_Asian_Paints.iloc[i]['Title'])

In [40]:
df2_clean_Asian_Paints

,CompanyName,Last_Fetch,Title
0,ASIAN PAINTS LTD,20201005,"buy asian paints target price rs two thousand,..."
1,ASIAN PAINTS LTD,20201006,market gains opening session hdfc asian paints...
2,ASIAN PAINTS LTD,20201007,technical update top pick technical pick asian...
3,ASIAN PAINTS LTD,20201008,technical update top picktechnical pick asian ...
4,ASIAN PAINTS LTD,20201009,imp results next fortnight tues 13th wiprowed ...
...,...,...,...
429,ASIAN PAINTS LTD,20230115,q3 results week icici bank hul asian paints js...
430,ASIAN PAINTS LTD,20230116,imp week earnings16th jan monday federal bank1...
431,ASIAN PAINTS LTD,20230118,earningswithetnow asian paints expected report...
432,ASIAN PAINTS LTD,20230119,asian paint hindustan zinc mahindra lifespace ...


In [41]:
df2_clean_Asian_Paints['level'] = level
df2_clean_Asian_Paints['overall_twitter_score'] = overall_twitter_score

<ipython-input-41-4d774f529bf0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['level'] = level
<ipython-input-41-4d774f529bf0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['overall_twitter_score'] = overall_twitter_score


In [42]:
df2_clean_Asian_Paints

,CompanyName,Last_Fetch,Title,level,overall_twitter_score
0,ASIAN PAINTS LTD,20201005,"buy asian paints target price rs two thousand,...",3 stars,0.291120
1,ASIAN PAINTS LTD,20201006,market gains opening session hdfc asian paints...,2 stars,0.292182
2,ASIAN PAINTS LTD,20201007,technical update top pick technical pick asian...,5 stars,0.319997
3,ASIAN PAINTS LTD,20201008,technical update top picktechnical pick asian ...,5 stars,0.319997
4,ASIAN PAINTS LTD,20201009,imp results next fortnight tues 13th wiprowed ...,5 stars,0.248748
...,...,...,...,...,...
429,ASIAN PAINTS LTD,20230115,q3 results week icici bank hul asian paints js...,5 stars,0.245013
430,ASIAN PAINTS LTD,20230116,imp week earnings16th jan monday federal bank1...,5 stars,0.248748
431,ASIAN PAINTS LTD,20230118,earningswithetnow asian paints expected report...,4 stars,0.274634
432,ASIAN PAINTS LTD,20230119,asian paint hindustan zinc mahindra lifespace ...,4 stars,0.318707


In [43]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as SIA
sia = SIA()
print(sia.polarity_scores('buy asian paints target price rs two thousand, one hundred and forty motilal oswal etmarkets dailyrecos stockpicks dailypicks asian paints home cor foray add sheen near-term volume growthimharshaj asian paints ltd consider q2 h1 fy21 results interim dividend oct twenty-two 2020asianpaints q2fy21 h1fy21 results interimdividend'))

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [44]:
texts2 = df2_clean_Asian_Paints['Title'].tolist()
negative_scores = []
neutral_scores = []
positive_scores = []
compound_scores = []
final_tag = []

In [45]:
for text in texts2:
    score_list = sia.polarity_scores(text)
    print(score_list)
    negative_scores.append(score_list['neg'])
    positive_scores.append(score_list['pos'])
    neutral_scores.append(score_list['neu'])
    compound_scores.append(score_list['compound'])
    if score_list['compound']>0:
        final_tag.append('POSITIVE')
    elif score_list['compound']<0:
        final_tag.append('NEGATIVE')
    else:
        final_tag.append('NEUTRAL')

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.956, 'pos': 0.044, 'compound': 0.34}
{'neg': 0.0, 'neu': 0.955, 'pos': 0.045, 'compound': 0.5267}
{'neg': 0.0, 'neu': 0.849, 'pos': 0.151, 'compound': 0.8834}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.688, 'pos': 0.312, 'compound': 0.7783}
{'neg': 0.0, 'neu': 0.917, 'pos': 0.083, 'compound': 0.2023}
{'neg': 0.103, 'neu': 0.816, 'pos': 0.082, 'compound': -0.1779}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.689, 'pos': 0.311, 'compound': 0.9477}
{'neg': 0.024, 'neu': 0.821, 'pos': 0.155, 'compound': 0.9966}
{'neg': 0.027, 'neu': 0.808, 'pos': 0.165, 'compound': 0.9903}
{'neg': 0.017, 'neu': 0.785, 'pos': 0.198, 'compound': 0.9816}
{'neg': 0.0, 'neu': 0.824, 'pos': 0.176, 'compound': 0.7906}
{'neg': 0.064, 'neu': 0.789, 'pos': 0.147, 'compound': 0.5267}
{'neg': 0.0, 'neu': 0.632, 'pos': 0.368, 'c

In [46]:
df2_clean_Asian_Paints['negative_score'] = negative_scores
df2_clean_Asian_Paints['positive_score'] = positive_scores
df2_clean_Asian_Paints['neutral_score'] = neutral_scores
df2_clean_Asian_Paints['compound_score'] = compound_scores
df2_clean_Asian_Paints['final_tag'] = final_tag

<ipython-input-46-40681d576903>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['negative_score'] = negative_scores
<ipython-input-46-40681d576903>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['positive_score'] = positive_scores
<ipython-input-46-40681d576903>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [47]:
df1_stock = pd.read_csv('Asain Paint Price.csv')
df1_stock.head()

,Date,Close
0,12/31/2018 15:30:00,1373.05
1,1/1/2019 15:30:00,1371.55
2,1/2/2019 15:30:00,1383.30
3,1/3/2019 15:30:00,1388.30
4,1/4/2019 15:30:00,1385.85


In [48]:
df1_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033 entries, 0 to 1032
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1033 non-null   object 
 1   Close   1033 non-null   float64
dtypes: float64(1), object(1)
memory usage: 16.3+ KB


In [49]:
df1_stock['Date'] = pd.to_datetime(df1_stock['Date'],errors='coerce')
df1_stock['Date'] = df1_stock['Date'].dt.strftime('%Y-%m-%d')

In [50]:
df1_stock

,Date,Close
0,2018-12-31,1373.05
1,2019-01-01,1371.55
2,2019-01-02,1383.30
3,2019-01-03,1388.30
4,2019-01-04,1385.85
...,...,...
1028,2023-02-23,2705.95
1029,2023-02-24,2738.50
1030,2023-02-27,2749.90
1031,2023-02-28,2828.80


In [51]:
df2_clean_Asian_Paints

,CompanyName,Last_Fetch,Title,level,overall_twitter_score,negative_score,positive_score,neutral_score,compound_score,final_tag
0,ASIAN PAINTS LTD,20201005,"buy asian paints target price rs two thousand,...",3 stars,0.291120,0.000,0.000,1.000,0.0000,NEUTRAL
1,ASIAN PAINTS LTD,20201006,market gains opening session hdfc asian paints...,2 stars,0.292182,0.000,0.044,0.956,0.3400,POSITIVE
2,ASIAN PAINTS LTD,20201007,technical update top pick technical pick asian...,5 stars,0.319997,0.000,0.045,0.955,0.5267,POSITIVE
3,ASIAN PAINTS LTD,20201008,technical update top picktechnical pick asian ...,5 stars,0.319997,0.000,0.151,0.849,0.8834,POSITIVE
4,ASIAN PAINTS LTD,20201009,imp results next fortnight tues 13th wiprowed ...,5 stars,0.248748,0.000,0.000,1.000,0.0000,NEUTRAL
...,...,...,...,...,...,...,...,...,...,...
429,ASIAN PAINTS LTD,20230115,q3 results week icici bank hul asian paints js...,5 stars,0.245013,0.000,0.055,0.945,0.3612,POSITIVE
430,ASIAN PAINTS LTD,20230116,imp week earnings16th jan monday federal bank1...,5 stars,0.248748,0.000,0.000,1.000,0.0000,NEUTRAL
431,ASIAN PAINTS LTD,20230118,earningswithetnow asian paints expected report...,4 stars,0.274634,0.074,0.177,0.749,0.9062,POSITIVE
432,ASIAN PAINTS LTD,20230119,asian paint hindustan zinc mahindra lifespace ...,4 stars,0.318707,0.029,0.119,0.852,0.9957,POSITIVE


In [52]:
df2_clean_Asian_Paints.rename(columns={'Last_Fetch':'Date'},inplace=True)

C:\Users\chinu\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [53]:
df2_clean_Asian_Paints.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434 entries, 0 to 433
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CompanyName            434 non-null    object 
 1   Date                   434 non-null    object 
 2   Title                  434 non-null    object 
 3   level                  434 non-null    object 
 4   overall_twitter_score  434 non-null    float64
 5   negative_score         434 non-null    float64
 6   positive_score         434 non-null    float64
 7   neutral_score          434 non-null    float64
 8   compound_score         434 non-null    float64
 9   final_tag              434 non-null    object 
dtypes: float64(5), object(5)
memory usage: 53.5+ KB


In [54]:
df2_clean_Asian_Paints

,CompanyName,Date,Title,level,overall_twitter_score,negative_score,positive_score,neutral_score,compound_score,final_tag
0,ASIAN PAINTS LTD,20201005,"buy asian paints target price rs two thousand,...",3 stars,0.291120,0.000,0.000,1.000,0.0000,NEUTRAL
1,ASIAN PAINTS LTD,20201006,market gains opening session hdfc asian paints...,2 stars,0.292182,0.000,0.044,0.956,0.3400,POSITIVE
2,ASIAN PAINTS LTD,20201007,technical update top pick technical pick asian...,5 stars,0.319997,0.000,0.045,0.955,0.5267,POSITIVE
3,ASIAN PAINTS LTD,20201008,technical update top picktechnical pick asian ...,5 stars,0.319997,0.000,0.151,0.849,0.8834,POSITIVE
4,ASIAN PAINTS LTD,20201009,imp results next fortnight tues 13th wiprowed ...,5 stars,0.248748,0.000,0.000,1.000,0.0000,NEUTRAL
...,...,...,...,...,...,...,...,...,...,...
429,ASIAN PAINTS LTD,20230115,q3 results week icici bank hul asian paints js...,5 stars,0.245013,0.000,0.055,0.945,0.3612,POSITIVE
430,ASIAN PAINTS LTD,20230116,imp week earnings16th jan monday federal bank1...,5 stars,0.248748,0.000,0.000,1.000,0.0000,NEUTRAL
431,ASIAN PAINTS LTD,20230118,earningswithetnow asian paints expected report...,4 stars,0.274634,0.074,0.177,0.749,0.9062,POSITIVE
432,ASIAN PAINTS LTD,20230119,asian paint hindustan zinc mahindra lifespace ...,4 stars,0.318707,0.029,0.119,0.852,0.9957,POSITIVE


In [55]:
df2_clean_Asian_Paints['Date'] = pd.to_datetime(df2_clean_Asian_Paints['Date'],errors='coerce')
df2_clean_Asian_Paints['Date'] = df2_clean_Asian_Paints['Date'].dt.strftime('%Y-%m-%d')

<ipython-input-55-44b20c1da776>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['Date'] = pd.to_datetime(df2_clean_Asian_Paints['Date'],errors='coerce')
<ipython-input-55-44b20c1da776>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_clean_Asian_Paints['Date'] = df2_clean_Asian_Paints['Date'].dt.strftime('%Y-%m-%d')


In [56]:
df2_clean_Asian_Paints

,CompanyName,Date,Title,level,overall_twitter_score,negative_score,positive_score,neutral_score,compound_score,final_tag
0,ASIAN PAINTS LTD,2020-10-05,"buy asian paints target price rs two thousand,...",3 stars,0.291120,0.000,0.000,1.000,0.0000,NEUTRAL
1,ASIAN PAINTS LTD,2020-10-06,market gains opening session hdfc asian paints...,2 stars,0.292182,0.000,0.044,0.956,0.3400,POSITIVE
2,ASIAN PAINTS LTD,2020-10-07,technical update top pick technical pick asian...,5 stars,0.319997,0.000,0.045,0.955,0.5267,POSITIVE
3,ASIAN PAINTS LTD,2020-10-08,technical update top picktechnical pick asian ...,5 stars,0.319997,0.000,0.151,0.849,0.8834,POSITIVE
4,ASIAN PAINTS LTD,2020-10-09,imp results next fortnight tues 13th wiprowed ...,5 stars,0.248748,0.000,0.000,1.000,0.0000,NEUTRAL
...,...,...,...,...,...,...,...,...,...,...
429,ASIAN PAINTS LTD,2023-01-15,q3 results week icici bank hul asian paints js...,5 stars,0.245013,0.000,0.055,0.945,0.3612,POSITIVE
430,ASIAN PAINTS LTD,2023-01-16,imp week earnings16th jan monday federal bank1...,5 stars,0.248748,0.000,0.000,1.000,0.0000,NEUTRAL
431,ASIAN PAINTS LTD,2023-01-18,earningswithetnow asian paints expected report...,4 stars,0.274634,0.074,0.177,0.749,0.9062,POSITIVE
432,ASIAN PAINTS LTD,2023-01-19,asian paint hindustan zinc mahindra lifespace ...,4 stars,0.318707,0.029,0.119,0.852,0.9957,POSITIVE


In [57]:
df2_clean_Asian_Paints.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434 entries, 0 to 433
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CompanyName            434 non-null    object 
 1   Date                   434 non-null    object 
 2   Title                  434 non-null    object 
 3   level                  434 non-null    object 
 4   overall_twitter_score  434 non-null    float64
 5   negative_score         434 non-null    float64
 6   positive_score         434 non-null    float64
 7   neutral_score          434 non-null    float64
 8   compound_score         434 non-null    float64
 9   final_tag              434 non-null    object 
dtypes: float64(5), object(5)
memory usage: 53.5+ KB


In [58]:
df1_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1033 entries, 0 to 1032
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1033 non-null   object 
 1   Close   1033 non-null   float64
dtypes: float64(1), object(1)
memory usage: 16.3+ KB


In [59]:
from functools import reduce
data_frames = [df2_clean_Asian_Paints,df1_stock]
df_stock_final_tweet = reduce(lambda  left,right: pd.merge(left,right,on=['Date'], how='inner'), data_frames)

In [60]:
df_stock_final_tweet

,CompanyName,Date,Title,level,overall_twitter_score,negative_score,positive_score,neutral_score,compound_score,final_tag,Close
0,ASIAN PAINTS LTD,2020-10-05,"buy asian paints target price rs two thousand,...",3 stars,0.291120,0.000,0.000,1.000,0.0000,NEUTRAL,2028.00
1,ASIAN PAINTS LTD,2020-10-06,market gains opening session hdfc asian paints...,2 stars,0.292182,0.000,0.044,0.956,0.3400,POSITIVE,2090.00
2,ASIAN PAINTS LTD,2020-10-07,technical update top pick technical pick asian...,5 stars,0.319997,0.000,0.045,0.955,0.5267,POSITIVE,2105.00
3,ASIAN PAINTS LTD,2020-10-08,technical update top picktechnical pick asian ...,5 stars,0.319997,0.000,0.151,0.849,0.8834,POSITIVE,2088.65
4,ASIAN PAINTS LTD,2020-10-09,imp results next fortnight tues 13th wiprowed ...,5 stars,0.248748,0.000,0.000,1.000,0.0000,NEUTRAL,2050.60
...,...,...,...,...,...,...,...,...,...,...,...
376,ASIAN PAINTS LTD,2023-01-13,bombay high court quashes reassessment order a...,3 stars,0.291120,0.000,0.000,1.000,0.0000,NEUTRAL,2909.15
377,ASIAN PAINTS LTD,2023-01-16,imp week earnings16th jan monday federal bank1...,5 stars,0.248748,0.000,0.000,1.000,0.0000,NEUTRAL,2916.05
378,ASIAN PAINTS LTD,2023-01-18,earningswithetnow asian paints expected report...,4 stars,0.274634,0.074,0.177,0.749,0.9062,POSITIVE,2945.25
379,ASIAN PAINTS LTD,2023-01-19,asian paint hindustan zinc mahindra lifespace ...,4 stars,0.318707,0.029,0.119,0.852,0.9957,POSITIVE,2866.15


In [61]:
df_stock_final_tweet.replace('stars', '', regex=True, inplace=True) 

In [62]:
df_stock_final_tweet['level'].value_counts()

4         177
3         120
5          57
2          25
1 star      2
Name: level, dtype: int64

In [63]:
df_stock_final_tweet.replace('star', '', regex=True, inplace=True) 

In [64]:
df_stock_final_tweet['level'].value_counts()

4     177
3     120
5      57
2      25
1       2
Name: level, dtype: int64

In [65]:
df_stock_final_tweet['level']=df_stock_final_tweet['level'].astype('int64')

In [66]:
final_tag_stock5=[]

for i in range(0, len(df_stock_final_tweet)-1):
    if df_stock_final_tweet.iloc[i+1]['Close']>df_stock_final_tweet.iloc[i]['Close']:
        final_tag_stock5.append('POSITIVE')
    elif df_stock_final_tweet.iloc[i+1]['Close']<df_stock_final_tweet.iloc[i]['Close']:
        final_tag_stock5.append('NEGATIVE')
    else:
        final_tag_stock5.append('NEUTRAL')

In [67]:
# giving the item to be inserted
insertItem = "NEGATIVE"

# giving the index value at which the item to be inserted
indexValue = 0

# inserting the given list item at the specified index(here 2)
final_tag_stock5.insert(indexValue, insertItem)

#final_tag_stock3.append('POSITIVE')

In [68]:
df_stock_final_tweet['final_tag_stock5']=final_tag_stock5

In [69]:
df_stock_final_tweet

,CompanyName,Date,Title,level,overall_twitter_score,negative_score,positive_score,neutral_score,compound_score,final_tag,Close,final_tag_stock5
0,ASIAN PAINTS LTD,2020-10-05,"buy asian paints target price rs two thousand,...",3,0.291120,0.000,0.000,1.000,0.0000,NEUTRAL,2028.00,NEGATIVE
1,ASIAN PAINTS LTD,2020-10-06,market gains opening session hdfc asian paints...,2,0.292182,0.000,0.044,0.956,0.3400,POSITIVE,2090.00,POSITIVE
2,ASIAN PAINTS LTD,2020-10-07,technical update top pick technical pick asian...,5,0.319997,0.000,0.045,0.955,0.5267,POSITIVE,2105.00,POSITIVE
3,ASIAN PAINTS LTD,2020-10-08,technical update top picktechnical pick asian ...,5,0.319997,0.000,0.151,0.849,0.8834,POSITIVE,2088.65,NEGATIVE
4,ASIAN PAINTS LTD,2020-10-09,imp results next fortnight tues 13th wiprowed ...,5,0.248748,0.000,0.000,1.000,0.0000,NEUTRAL,2050.60,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...
376,ASIAN PAINTS LTD,2023-01-13,bombay high court quashes reassessment order a...,3,0.291120,0.000,0.000,1.000,0.0000,NEUTRAL,2909.15,NEGATIVE
377,ASIAN PAINTS LTD,2023-01-16,imp week earnings16th jan monday federal bank1...,5,0.248748,0.000,0.000,1.000,0.0000,NEUTRAL,2916.05,POSITIVE
378,ASIAN PAINTS LTD,2023-01-18,earningswithetnow asian paints expected report...,4,0.274634,0.074,0.177,0.749,0.9062,POSITIVE,2945.25,POSITIVE
379,ASIAN PAINTS LTD,2023-01-19,asian paint hindustan zinc mahindra lifespace ...,4,0.318707,0.029,0.119,0.852,0.9957,POSITIVE,2866.15,NEGATIVE


In [70]:
final_tag_bert_tweet=[]

for i in range(0, len(df_stock_final_tweet)):
    if df_stock_final_tweet.iloc[i]['level']>2:
        final_tag_bert_tweet.append('POSITIVE')
    elif df_stock_final_tweet.iloc[i]['level']<=2:
        final_tag_bert_tweet.append('NEGATIVE')
    else:
        final_tag_bert_tweet.append('NEUTRAL')

In [71]:
 len(final_tag_bert_tweet)

381

In [72]:
df_stock_final_tweet['final_tag_bert']=final_tag_bert_tweet

In [73]:
from sklearn.metrics import classification_report as crep
print("sentiment analysis performance for bert:")
print(crep(df_stock_final_tweet['final_tag_stock5'],df_stock_final_tweet['final_tag_bert']))

sentiment analysis performance for bert:
              precision    recall  f1-score   support

    NEGATIVE       0.37      0.05      0.09       188
    POSITIVE       0.50      0.91      0.64       193

    accuracy                           0.49       381
   macro avg       0.43      0.48      0.37       381
weighted avg       0.43      0.49      0.37       381



In [74]:
#df_stock_final_tweet1=df_stock_final_tweet.loc[df_stock_final_tweet['final_tag']!='NEUTRAL']

In [75]:
from sklearn.metrics import classification_report as crep
print("sentiment analysis performance for vader:")
print(crep(df_stock_final_tweet.loc[df_stock_final_tweet['final_tag']!='NEUTRAL']['final_tag_stock5'],df_stock_final_tweet.loc[df_stock_final_tweet['final_tag']!='NEUTRAL']['final_tag']))

sentiment analysis performance for vader:
              precision    recall  f1-score   support

    NEGATIVE       0.44      0.22      0.29       151
    POSITIVE       0.51      0.74      0.60       163

    accuracy                           0.49       314
   macro avg       0.47      0.48      0.45       314
weighted avg       0.47      0.49      0.45       314



In [77]:
file_name="Twitter_score.csv"
df_stock_final_tweet.to_csv(file_name)